# Apps Portfolio

the goal is to gather data and perform analysis on performance

In [60]:
from csv import reader

opened_file = open('AppleStore.csv')
read_file = reader(opened_file)
apple_apps_data = list(read_file)

opened_file = open('googleplaystore.csv')
read_file = reader(opened_file)
google_apps_data = list(read_file)
del(google_apps_data[10473])

In [61]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

In [62]:
explore_data(apple_apps_data, 0, 2, True)
print('')
explore_data(google_apps_data, 0, 2, True)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


Number of rows: 7198
Number of columns: 16


['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


Number of rows: 10841
Number of columns: 13


There could potentially be duplications in the data
Let's run a check to see how many dupliations there are

In [63]:
duplicate_apps = []
unique_apps = []

for apps in google_apps_data[1:]:
    name = apps[0]
    if name in unique_apps:
        duplicate_apps.append(name)
    else: 
        unique_apps.append(name)
        
print('Dupilcate Google Apps: ', len(duplicate_apps))
print('Unique Google Apps: ' , len(unique_apps))

duplicate_apps = []
unique_apps = []

for apps in apple_apps_data[1:]:
    name = apps[1]
    if name in unique_apps:
        duplicate_apps.append(name)
    else: 
        unique_apps.append(name)
        
print('Dupilcate Apple Apps: ', len(duplicate_apps))
print('Unique Apple Apps: ' , len(unique_apps))

Dupilcate Google Apps:  1181
Unique Google Apps:  9659
Dupilcate Apple Apps:  2
Unique Apple Apps:  7195


We'll remove the duplicate apps and only keep the record with the highest number of reviews, assuming it's the most recent recording of the data

In [64]:
review_max = {}
for apps in google_apps_data[1:]:
    name = apps[0]
    n_reviews = int(apps[3])
    if (name in review_max and review_max[name] < n_reviews):
        review_max[name] = n_reviews
    elif name not in review_max: 
        review_max[name] = n_reviews

android_clean = []
already_added = []
for apps in google_apps_data[1:]:
    name = apps[0]
    n_reviews = int(apps[3])
    if (n_reviews == review_max[name] and name not in already_added): 
        android_clean.append(apps)
        already_added.append(name)
        
print('Cleaned Google Apps: ', len(android_clean))

review_max = {}
for apps in apple_apps_data[1:]:
    name = apps[1]
    n_reviews = int(apps[5])
    if (name in review_max and review_max[name] < n_reviews):
        review_max[name] = n_reviews
    elif name not in review_max: 
        review_max[name] = n_reviews

apple_clean = []
already_added = []
for apps in apple_apps_data[1:]:
    name = apps[1]
    n_reviews = int(apps[5])
    if (n_reviews == review_max[name] and name not in already_added): 
        apple_clean.append(apps)
        already_added.append(name)
        
print('Cleaned Apple Apps: ', len(apple_clean))

Cleaned Google Apps:  9659
Cleaned Apple Apps:  7195


In [65]:
def is_english(title):
    non_english_character = 0
    for character in title:
        if ord(character) > 127:
            non_english_character += 1
    if non_english_character > 3: 
        return False
    else: 
        return True

In [66]:
'''
print(is_english('Instagram'))
print(is_english('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(is_english('Docs To Go™ Free Office Suite'))
print(is_english('Instachat 😜'))
'''

"\nprint(is_english('Instagram'))\nprint(is_english('爱奇艺PPS -《欢乐颂2》电视剧热播'))\nprint(is_english('Docs To Go™ Free Office Suite'))\nprint(is_english('Instachat 😜'))\n"

In [67]:
apple_clean_english = []
for apps in apple_clean:
    name = apps[1]
    if (is_english(name)): 
        apple_clean_english.append(apps)

android_clean_english = []
for apps in android_clean:
    name = apps[0]
    if (is_english(name)): 
        android_clean_english.append(apps)

store only free apps in a separate list

checking to see if $ sign equals non-free

In [68]:
android_apps_with_ds = 0
android_apps_with_ds_ninety_nine = 0
for apps in android_clean_english:
    price = apps[7]
    if price[0:3] == '$0.':
        android_apps_with_ds += 1
for apps in android_clean_english:
    price = apps[7]
    if price[0:5] == '$0.99':
        android_apps_with_ds_ninety_nine += 1
        
print(android_apps_with_ds)
print(android_apps_with_ds_ninety_nine)

145
145


yes it does! - we could have used str.replace(old, new) here

In [70]:
apple_clean_english_free = []
for apps in apple_clean_english:
    price = float(apps[4])
    if price == 0.0:
        apple_clean_english_free.append(apps)

android_clean_english_free = []
for apps in android_clean_english:
    price = apps[7]
    if price[0] != '$':
        price = float(price)
        if price == 0.0:
            android_clean_english_free.append(apps)

create some frequency tables to determine popular genres of apps

In [78]:
apple_freq_genres = {}
for apps in apple_clean_english_free: 
    genre = apps[11]
    if genre in apple_freq_genres:
        apple_freq_genres[genre] += 1
    else: 
        apple_freq_genres[genre] = 1
for genre in apple_freq_genres:
    print(genre + ': ' + str(apple_freq_genres[genre]))

print('')
android_freq_genres = {}
for apps in android_clean_english_free: 
    genre = apps[1]
    if genre in android_freq_genres:
        android_freq_genres[genre] += 1
    else: 
        android_freq_genres[genre] = 1
for genre in android_freq_genres:
    print(genre + ': ' + str(android_freq_genres[genre]))

Medical: 6
Weather: 28
Reference: 18
Travel: 40
Photo & Video: 160
Sports: 69
Catalogs: 4
Book: 14
Shopping: 84
Health & Fitness: 65
Music: 66
Finance: 36
Education: 118
Utilities: 81
News: 43
Navigation: 6
Lifestyle: 51
Entertainment: 254
Games: 1872
Food & Drink: 26
Business: 17
Productivity: 56
Social Networking: 106


WEATHER: 71
HOUSE_AND_HOME: 73
BEAUTY: 53
FINANCE: 328
SOCIAL: 236
TOOLS: 750
COMMUNICATION: 287
SPORTS: 301
VIDEO_PLAYERS: 159
FAMILY: 1676
SHOPPING: 199
LIFESTYLE: 346
BUSINESS: 407
PARENTING: 58
MEDICAL: 313
PRODUCTIVITY: 345
LIBRARIES_AND_DEMO: 83
NEWS_AND_MAGAZINES: 248
DATING: 165
TRAVEL_AND_LOCAL: 207
MAPS_AND_NAVIGATION: 124
GAME: 862
FOOD_AND_DRINK: 110
AUTO_AND_VEHICLES: 82
EVENTS: 63
HEALTH_AND_FITNESS: 273
BOOKS_AND_REFERENCE: 190
ART_AND_DESIGN: 57
COMICS: 55
ENTERTAINMENT: 85
PERSONALIZATION: 294
PHOTOGRAPHY: 261
EDUCATION: 103


In [86]:
def freq_table(dataset, index):
    freq_dictionary = {}
    for each_row in dataset:
        if each_row[index] in freq_dictionary: 
            freq_dictionary[each_row[index]] += 1
        else: 
            freq_dictionary[each_row[index]] = 1
    for each_element in freq_dictionary: 
        freq_dictionary[each_element] = round(freq_dictionary[each_element] / len(dataset) * 100, 2)
    return freq_dictionary

In [87]:
print(freq_table(android_clean_english_free, 1))

{'WEATHER': 0.8, 'HOUSE_AND_HOME': 0.82, 'BEAUTY': 0.6, 'FINANCE': 3.7, 'SOCIAL': 2.66, 'TOOLS': 8.46, 'COMMUNICATION': 3.24, 'SPORTS': 3.4, 'VIDEO_PLAYERS': 1.79, 'FAMILY': 18.91, 'SHOPPING': 2.25, 'LIFESTYLE': 3.9, 'BUSINESS': 4.59, 'PARENTING': 0.65, 'MEDICAL': 3.53, 'PRODUCTIVITY': 3.89, 'LIBRARIES_AND_DEMO': 0.94, 'NEWS_AND_MAGAZINES': 2.8, 'DATING': 1.86, 'TRAVEL_AND_LOCAL': 2.34, 'MAPS_AND_NAVIGATION': 1.4, 'GAME': 9.72, 'FOOD_AND_DRINK': 1.24, 'AUTO_AND_VEHICLES': 0.93, 'EVENTS': 0.71, 'HEALTH_AND_FITNESS': 3.08, 'BOOKS_AND_REFERENCE': 2.14, 'ART_AND_DESIGN': 0.64, 'COMICS': 0.62, 'ENTERTAINMENT': 0.96, 'PERSONALIZATION': 3.32, 'PHOTOGRAPHY': 2.94, 'EDUCATION': 1.16}


In [88]:
def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

In [95]:
display_table(apple_clean_english_free, 11)
print('')
display_table(android_clean_english_free, 1)
print('')
display_table(android_clean_english_free, 9)

Games : 58.14
Entertainment : 7.89
Photo & Video : 4.97
Education : 3.66
Social Networking : 3.29
Shopping : 2.61
Utilities : 2.52
Sports : 2.14
Music : 2.05
Health & Fitness : 2.02
Productivity : 1.74
Lifestyle : 1.58
News : 1.34
Travel : 1.24
Finance : 1.12
Weather : 0.87
Food & Drink : 0.81
Reference : 0.56
Business : 0.53
Book : 0.43
Navigation : 0.19
Medical : 0.19
Catalogs : 0.12

FAMILY : 18.91
GAME : 9.72
TOOLS : 8.46
BUSINESS : 4.59
LIFESTYLE : 3.9
PRODUCTIVITY : 3.89
FINANCE : 3.7
MEDICAL : 3.53
SPORTS : 3.4
PERSONALIZATION : 3.32
COMMUNICATION : 3.24
HEALTH_AND_FITNESS : 3.08
PHOTOGRAPHY : 2.94
NEWS_AND_MAGAZINES : 2.8
SOCIAL : 2.66
TRAVEL_AND_LOCAL : 2.34
SHOPPING : 2.25
BOOKS_AND_REFERENCE : 2.14
DATING : 1.86
VIDEO_PLAYERS : 1.79
MAPS_AND_NAVIGATION : 1.4
FOOD_AND_DRINK : 1.24
EDUCATION : 1.16
ENTERTAINMENT : 0.96
LIBRARIES_AND_DEMO : 0.94
AUTO_AND_VEHICLES : 0.93
HOUSE_AND_HOME : 0.82
WEATHER : 0.8
EVENTS : 0.71
PARENTING : 0.65
ART_AND_DESIGN : 0.64
COMICS : 0.62
BEAUTY

In [114]:
apple_genre_freq_table = freq_table(apple_clean_english_free, 11)
for genre in apple_genre_freq_table: 
    total = 0
    len_genre = 0
    for apps in apple_clean_english_free:
        genre_app = apps[11]
        if genre_app == genre:
            n_ratings = int(apps[5])
            total += n_ratings
            len_genre += 1
    print('Genre: ' + genre + ' has avg rating numbers of ' + str(round((total / len_genre), 2)))

Genre: Medical has avg rating numbers of 612.0
Genre: Weather has avg rating numbers of 52279.89
Genre: Reference has avg rating numbers of 74942.11
Genre: Travel has avg rating numbers of 28243.8
Genre: Photo & Video has avg rating numbers of 28441.54
Genre: Sports has avg rating numbers of 23008.9
Genre: Catalogs has avg rating numbers of 4004.0
Genre: Book has avg rating numbers of 39758.5
Genre: Shopping has avg rating numbers of 26919.69
Genre: Health & Fitness has avg rating numbers of 23298.02
Genre: Music has avg rating numbers of 57326.53
Genre: Finance has avg rating numbers of 31467.94
Genre: Education has avg rating numbers of 7003.98
Genre: Utilities has avg rating numbers of 18684.46
Genre: News has avg rating numbers of 21248.02
Genre: Navigation has avg rating numbers of 86090.33
Genre: Lifestyle has avg rating numbers of 16485.76
Genre: Entertainment has avg rating numbers of 14029.83
Genre: Games has avg rating numbers of 22812.92
Genre: Food & Drink has avg rating nu

In [ ]:
android_genre_freq_table = freq_table(android_clean_english_free, 1)
for category in android_genre_freq_table: 
    total = 0
    len_category = 0
    for apps in android_clean_english_free:
        category_app = apps[1]
        if category_app == genre:
            n_installs = int(apps[5].replace('+','').replace(',',''))
            total += n_ratings
            len_category += 1
    print('Genre: ' + genre + ' has avg rating numbers of ' + str(round((total / len_genre), 2)))